In [31]:

## Import libraries section ##

import numpy as np
import cv2
import os
import math
from LFR_utils import read_poses_and_images,pose_to_virtualcamera, init_aos, init_window
import LFR_utils as utils
import pyaos
import glm
import glob
import re


## path to where the results will be stored 

user_name = os.getlogin()

base_path = 'C:\\Users\\{}\\Documents\\computervision\\'.format(user_name)

print(f"Starting with base path: {base_path}")

Download_Location = base_path    ## Enter path to the directory where you want to save the results.
print(Download_Location)
Integral_Path = os.path.join(Download_Location,'integrals') # Note that your results will be saved to this integrals folder.

# Check if the directory already exists
if not os.path.exists(Integral_Path):
    os.mkdir(Integral_Path)
else:
    print(f"The directory '{Integral_Path}' already exists.")



Starting with base path: C:\Users\marti\Documents\computervision\
C:\Users\marti\Documents\computervision\
The directory 'C:\Users\marti\Documents\computervision\integrals' already exists.


In [32]:
#############################Start the AOS Renderer###############################################################
w,h,fovDegrees = 512, 512, 50 # # resolution and field of view. This should not be changed.
render_fov = 50

if 'window' not in locals() or window == None:
                                    
    window = pyaos.PyGlfwWindow( w, h, 'AOS' )  
     
aos = pyaos.PyAOS(w,h,fovDegrees) 


set_folder = base_path+r'\\ImageRestoration-CV\\LFR\\python'          # Enter path to your LFR/python directory
aos.loadDEM( os.path.join(set_folder,'zero_plane.obj'))

####################################################################################################################

In [33]:
#############################Create Poses for Initial Positions###############################################################

# Below are certain functions required to convert the poses to a certain format to be compatabile with the AOS Renderer.

def eul2rotm(theta) :
    s_1 = math.sin(theta[0])
    c_1 = math.cos(theta[0]) 
    s_2 = math.sin(theta[1]) 
    c_2 = math.cos(theta[1]) 
    s_3 = math.sin(theta[2]) 
    c_3 = math.cos(theta[2])
    rotm = np.identity(3)
    rotm[0,0] =  c_1*c_2
    rotm[0,1] =  c_1*s_2*s_3 - s_1*c_3
    rotm[0,2] =  c_1*s_2*c_3 + s_1*s_3

    rotm[1,0] =  s_1*c_2
    rotm[1,1] =  s_1*s_2*s_3 + c_1*c_3
    rotm[1,2] =  s_1*s_2*c_3 - c_1*s_3

    rotm[2,0] = -s_2
    rotm[2,1] =  c_2*s_3
    rotm[2,2] =  c_2*c_3        

    return rotm

def createviewmateuler(eulerang, camLocation):
    
    rotationmat = eul2rotm(eulerang)
    translVec =  np.reshape((-camLocation @ rotationmat),(3,1))
    conjoinedmat = (np.append(np.transpose(rotationmat), translVec, axis=1))
    return conjoinedmat

def divide_by_alpha(rimg2):
        a = np.stack((rimg2[:,:,3],rimg2[:,:,3],rimg2[:,:,3]),axis=-1)
        return rimg2[:,:,:3]/a

def pose_to_virtualcamera(vpose ):
    vp = glm.mat4(*np.array(vpose).transpose().flatten())
    #vp = vpose.copy()
    ivp = glm.inverse(glm.transpose(vp))
    #ivp = glm.inverse(vpose)
    Posvec = glm.vec3(ivp[3])
    Upvec = glm.vec3(ivp[1])
    FrontVec = glm.vec3(ivp[2])
    lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
    cameraviewarr = np.asarray(lookAt)
    #print(cameraviewarr)
    return cameraviewarr  

In [44]:
from tqdm import tqdm


class ImageGroup:
    def __init__(self, image_index):
        assert image_index is not None
        assert isinstance(image_index, int)
        assert image_index >= 0
        self.image_index = image_index
        self.filenames = []
        self.ground_truth = None
        self.parameter_file = None

    def add_filename(self, filename):
        self.filenames.append(filename)


########################## Below we generate the poses for rendering #####################################
# This is based on how renderer is implemented. 

def load_filenames(base_path):
    all_files = os.listdir(base_path)
    file_dict = {}

    # Regex to extract image_index and pose_index
    file_pattern = re.compile(r"([01])_(\d+)_pose_(\d+)_thermal\.png")

    for file in all_files:
        match = file_pattern.match(file)
        if match:
            image_index = int(match.group(2))
            if image_index not in file_dict:
                file_dict[image_index] = ImageGroup(image_index)
            file_dict[image_index].add_filename(os.path.join(base_path, file))
    return file_dict

def create_output_directory(base_output_path, part_name):
    output_dir = os.path.join(base_output_path, part_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

def create_integral_image(file_list, image_name, output_path, Focal_plane):

    assert len(file_list) == 11, f"Expected 11 images but got {len(file_list)}"
    Numberofimages = 11
    # ref_loc is the reference location or the poses of the images. The poses are the same for the dataset and therefore only the images have to be replaced.
    ref_loc = [[5,4,3,2,1,0,-1,-2,-3,-4,-5],[0,0,0,0,0,0,0,0,0,0,0]]   # These are the x and y positions of the images. It is of the form [[x_positions],[y_positions]]

    altitude_list = [35,35,35,35,35,35,35,35,35,35,35] # [Z values which is the height]

    center_index = 5  # this is important, this will be the pose index at which the integration should happen. For example if you have 5 images, lets say you want to integrate all 5 images to the second image position. Then your center_index is 1 as index starts from zero.

    site_poses = []
    for i in range(Numberofimages):
        EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
        NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
        M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i], - altitude_list[i]] ))
        #print('m',M)
        ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
        #print(ViewMatrix)
        camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
        #print(camerapose)
        site_poses.append(camerapose)  # site_poses is a list now containing all the poses of all the images in a certain format that is accecpted by the renderer.
        
    imagelist = []

    for file in file_list:
        img = cv2.imread(file)
        assert img is not None, f"Could not read image {file}"
        imagelist.append(img)
        
    #############################Read the generated images from the simulator and store in a list ###############################################################
        

    aos.clearViews()   # Every time you call the renderer you should use this line to clear the previous views  
    for i in range(len(imagelist)):
            aos.addView(imagelist[i], site_poses[i], "DEM BlobTrack")  # Here we are adding images to the renderer one by one.
    aos.setDEMTransform([0,0,Focal_plane])

    proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
    tmp_RGB = divide_by_alpha(proj_RGBimg)
    output_image_name = os.path.join(output_path, 'integral_{}.png'.format(image_name))
    #print("Writing image to {}".format(output_image_name))
    cv2.imwrite(output_image_name, tmp_RGB)   # Final result. Check the integral result in the integrals folder.


base_paths = ["Part1", "Part1 2"]
base_output_path = Integral_Path

for part_name in tqdm(base_paths, desc="Processing parts"):
    part_base_path = os.path.join(base_path, part_name)
    output_path = create_output_directory(base_output_path, part_name)

    file_dict = load_filenames(part_base_path)

    for image_index in range(2):
        image_idx_list = file_dict.keys()

        for image_idx in tqdm(image_idx_list, desc=f"Processing images in {part_name}"):
            img_group : ImageGroup = file_dict[image_idx]

            print(f"Processing image index {image_idx} with {len(img_group.filenames)} images : {img_group.filenames}")

            if(len(img_group.filenames) != 11):
                print(f"Skipping pose {image_index} because it has {len(img_group.filenames)} images")
                continue

            for focal_stack_img_index in tqdm(range(0, 31), desc=f"Generating focal stacks for image index {image_idx}"):
                create_integral_image(img_group.filenames, f'{image_index}_pose_{focal_stack_img_index}', output_path, image_index * 0.1)

Processing image index 0 with 11 images : ['C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_0_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_10_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_1_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_2_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_3_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_4_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_5_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_6_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_7_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_8_thermal.png', 'C:\\Users\\marti\\Documents\\computervision\\Part1\\0_0_pose_9_thermal.png']
Processing image index 1000 with 11 images : ['C:\\Users\\marti\\Documents\\computervision\\Part1\

KeyboardInterrupt: 